# Word2Vec Embedding

In [1]:
from gensim.models import KeyedVectors
import json
import numpy as np
import torch
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import copy

In [2]:
label_map = {'B', 'I', 'O'}

In [3]:
word2vec_model_path = '../GoogleNews-vectors-negative300.bin'
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)


with open('ATE_train.json', 'r') as file:
    dataset = json.load(file)

print(len(dataset))

max_seq_len = 85
word_embeddings = []
labels = []

for key, value in dataset.items():
    text = value['text']
    label_seq = value['labels']

    label_list = list(label_map)  
    label_indices = [label_list.index(label) for label in label_seq]
    label_indices_padded = label_indices[:max_seq_len] + [0] * (max_seq_len - len(label_indices))
    labels.append(label_indices_padded)

    vectors = [word2vec_model[word] if word in word2vec_model else np.zeros(300) for word in text] 
    vectors_padded = [np.pad(vec, (0, 300), constant_values=0)[:300] for vec in vectors]
    vectors_padded += [np.zeros(300)] * (max_seq_len - len(vectors_padded))
    word_embeddings.append(vectors_padded)

word_embeddings_array = np.array(word_embeddings)
print(word_embeddings_array.shape)
texts_tensor = torch.tensor(word_embeddings_array, dtype=torch.float32)
labels_tensor = torch.tensor(labels)

906
(906, 85, 300)


In [4]:
with open('ATE_val.json', 'r') as file:
    valset = json.load(file)

print(len(valset))
max_seq_len = 85

word_embeddings = []
labels = []

for key, value in valset.items():
    text = value['text']
    label_seq = value['labels']

    label_list = list(label_map)  # Store index of each 27 classes created
    label_indices = [label_list.index(label) for label in label_seq]
    label_indices_padded = label_indices[:max_seq_len] + [0] * (max_seq_len - len(label_indices))
    labels.append(label_indices_padded)

    vectors = [word2vec_model[word] if word in word2vec_model else np.zeros(300) for word in text]
    vectors_padded = [np.pad(vec, (0, 300), constant_values=0)[:300] for vec in vectors]
    vectors_padded += [np.zeros(300)] * (max_seq_len - len(vectors_padded))
    word_embeddings.append(vectors_padded)

word_embeddings_array = np.array(word_embeddings)

test_texts_tensor = torch.tensor(word_embeddings_array, dtype=torch.float32)
test_labels_tensor = torch.tensor(labels)

219


In [ ]:
# import wandb
# wandb.login()

In [ ]:
# wandb.init(
#     project="nlp_ass2B", 
#     name=f"RNN_Word2Vec"
# )

In [5]:
class RNNTagger(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNTagger, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out)
        return out

input_size = 300
hidden_size = 256
output_size = 3

model = RNNTagger(input_size, hidden_size, output_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 45
batch_size = 32 

texts_tensor = texts_tensor.to(device)
labels_tensor = labels_tensor.to(device)
val_texts_tensor = test_texts_tensor
val_labels_tensor =  test_labels_tensor
val_texts_tensor = val_texts_tensor.to(device)
val_labels_tensor = val_labels_tensor.to(device)

save_loss = 0

for epoch in range(num_epochs):
    
    model.train()  
    total_loss=0
    train_predictions = []
    train_labels = []
    for i in range(0, len(texts_tensor), batch_size):
        optimizer.zero_grad()
        batch_texts = texts_tensor[i:i+batch_size]
        batch_labels = labels_tensor[i:i+batch_size].view(-1)
        outputs = model(batch_texts)
        loss = criterion(outputs.view(-1, output_size), batch_labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        train_predictions.extend(torch.argmax(outputs, dim=2).flatten().cpu().tolist())
        train_labels.extend(batch_labels.cpu().tolist())
        
    train_f1 = f1_score(train_labels, train_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_loss/len(texts_tensor):.4f}, Training F1 Score: {train_f1:.4f}')
    
    # Validation
    model.eval()  
    total_val_loss = 0
    val_predictions = []
    val_labels = []
    
    with torch.no_grad():
        for i in range(0, len(val_texts_tensor), batch_size):
            batch_texts_val = val_texts_tensor[i:i+batch_size]
            batch_labels_val = val_labels_tensor[i:i+batch_size].view(-1)
            outputs_val = model(batch_texts_val)
            val_loss = criterion(outputs_val.view(-1, output_size), batch_labels_val)
            total_val_loss += val_loss.item()   
            val_predictions.extend(torch.argmax(outputs_val, dim=2).flatten().cpu().tolist())
            val_labels.extend(batch_labels_val.cpu().tolist())
    
    val_f1 = f1_score(val_labels, val_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {total_val_loss/len(val_texts_tensor):.4f}, Validation F1 Score: {val_f1:.4f}')
    if(val_f1>save_loss):
        save_loss = val_f1
        torch.save(copy.deepcopy(model.state_dict()),'t2_rnn_word2vec.pt')
    log_metric = {"Epoch": epoch+1, "Training Loss": total_loss/len(texts_tensor), "Training F1 Score": train_f1, "Validation Loss": total_val_loss/len(val_texts_tensor), "Validation F1 Score": val_f1}
    # wandb.log(log_metric)
print("Finished Training")
    

/home/iiitd/miniconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch [1/45], Training Loss: 0.0165, Training F1 Score: 0.4911
Epoch [1/45], Validation Loss: 0.0090, Validation F1 Score: 0.5955
Epoch [2/45], Training Loss: 0.0052, Training F1 Score: 0.6266
Epoch [2/45], Validation Loss: 0.0040, Validation F1 Score: 0.6526
Epoch [3/45], Training Loss: 0.0036, Training F1 Score: 0.6690
Epoch [3/45], Validation Loss: 0.0034, Validation F1 Score: 0.6949
Epoch [4/45], Training Loss: 0.0032, Training F1 Score: 0.7000
Epoch [4/45], Validation Loss: 0.0030, Validation F1 Score: 0.7394
Epoch [5/45], Training Loss: 0.0029, Training F1 Score: 0.7428
Epoch [5/45], Validation Loss: 0.0028, Validation F1 Score: 0.7665
Epoch [6/45], Training Loss: 0.0027, Training F1 Score: 0.7684
Epoch [6/45], Validation Loss: 0.0026, Validation F1 Score: 0.7836
Epoch [7/45], Training Loss: 0.0026, Training F1 Score: 0.7842
Epoch [7/45], Validation Loss: 0.0025, Validation F1 Score: 0.7986
Epoch [8/45], Training Loss: 0.0024, Training F1 Score: 0.7972
Epoch [8/45], Validation Lo

In [7]:
with open('ATE_test.json', 'r') as file:
    valset = json.load(file)

print(len(valset))
max_seq_len = 85

word_embeddings = []
labels = []

for key, value in valset.items():
    text = value['text']
    label_seq = value['labels']

    label_list = list(label_map)  # Store index of each 27 classes created
    label_indices = [label_list.index(label) for label in label_seq]
    label_indices_padded = label_indices[:max_seq_len] + [0] * (max_seq_len - len(label_indices))
    labels.append(label_indices_padded)

    vectors = [word2vec_model[word] if word in word2vec_model else np.zeros(300) for word in text]
    vectors_padded = [np.pad(vec, (0, 300), constant_values=0)[:300] for vec in vectors]
    vectors_padded += [np.zeros(300)] * (max_seq_len - len(vectors_padded))
    word_embeddings.append(vectors_padded)

word_embeddings_array = np.array(word_embeddings)

lol_texts_tensor = torch.tensor(word_embeddings_array, dtype=torch.float32)
lol_labels_tensor = torch.tensor(labels)

with torch.no_grad():
    model = RNNTagger(input_size, hidden_size, output_size)
    model = model.to(device)
    model.load_state_dict(torch.load('t2_rnn_word2vec.pt'))
    model.eval()
    test_outputs = model(lol_texts_tensor.to(device)) 
    argmax_indices = torch.argmax(test_outputs, dim=2)
    reshaped_tensor = argmax_indices.view(328, 85)
    fl_out = reshaped_tensor.flatten()
    fl_label = lol_labels_tensor.flatten().to(device) 
    f1 = f1_score(fl_out.cpu(), fl_label.cpu(), average='macro') 
    print("F1 Score on Test Set:", f1)
    accuracy = accuracy_score(fl_out.cpu(), fl_label.cpu())
    print("Accuracy on Test Set:", accuracy)

328
F1 Score on Test Set: 0.8795341934715708
Accuracy on Test Set: 0.9773672883787662


In [ ]:
# wandb.init(
#     project="nlp_ass2B", 
#     name=f"LSTM_Word2Vec"
# )

In [9]:
class LSTMTagger(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMTagger, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

input_size = 300
hidden_size = 256
output_size = 3

model = LSTMTagger(input_size, hidden_size, output_size) # Move model to GPU if available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 45
batch_size = 32 

texts_tensor = texts_tensor.to(device)
labels_tensor = labels_tensor.to(device)
val_texts_tensor = test_texts_tensor
val_labels_tensor =  test_labels_tensor
val_texts_tensor = val_texts_tensor.to(device)
val_labels_tensor = val_labels_tensor.to(device)

save_loss = 0

for epoch in range(num_epochs):
    
    model.train()  
    total_loss=0
    train_predictions = []
    train_labels = []
    for i in range(0, len(texts_tensor), batch_size):
        optimizer.zero_grad()
        batch_texts = texts_tensor[i:i+batch_size]
        batch_labels = labels_tensor[i:i+batch_size].view(-1)
        outputs = model(batch_texts)
        loss = criterion(outputs.view(-1, output_size), batch_labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        train_predictions.extend(torch.argmax(outputs, dim=2).flatten().cpu().tolist())
        train_labels.extend(batch_labels.cpu().tolist())
        
    train_f1 = f1_score(train_labels, train_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_loss/len(texts_tensor):.4f}, Training F1 Score: {train_f1:.4f}')
    
    # Validation
    model.eval()  
    total_val_loss = 0
    val_predictions = []
    val_labels = []
    
    with torch.no_grad():
        for i in range(0, len(val_texts_tensor), batch_size):
            batch_texts_val = val_texts_tensor[i:i+batch_size]
            batch_labels_val = val_labels_tensor[i:i+batch_size].view(-1)
            outputs_val = model(batch_texts_val)
            val_loss = criterion(outputs_val.view(-1, output_size), batch_labels_val)
            total_val_loss += val_loss.item()
            val_predictions.extend(torch.argmax(outputs_val, dim=2).flatten().cpu().tolist())
            val_labels.extend(batch_labels_val.cpu().tolist())
    
    val_f1 = f1_score(val_labels, val_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {total_val_loss/len(val_texts_tensor):.4f}, Validation F1 Score: {val_f1:.4f}')
    if(val_f1>save_loss):
        save_loss = val_f1
        torch.save(model.state_dict(),'t2_lstm_word2vec.pt')
    log_metric = {"Epoch": epoch+1, "Training Loss": total_loss/len(texts_tensor), "Training F1 Score": train_f1, "Validation Loss": total_val_loss/len(val_texts_tensor), "Validation F1 Score": val_f1}
    # wandb.log(log_metric)
print("Finished Training")
    

Epoch [1/45], Training Loss: 0.0146, Training F1 Score: 0.5702
Epoch [1/45], Validation Loss: 0.0054, Validation F1 Score: 0.6169
Epoch [2/45], Training Loss: 0.0047, Training F1 Score: 0.6268
Epoch [2/45], Validation Loss: 0.0037, Validation F1 Score: 0.6345
Epoch [3/45], Training Loss: 0.0036, Training F1 Score: 0.6361
Epoch [3/45], Validation Loss: 0.0033, Validation F1 Score: 0.6368
Epoch [4/45], Training Loss: 0.0032, Training F1 Score: 0.6390
Epoch [4/45], Validation Loss: 0.0030, Validation F1 Score: 0.6438
Epoch [5/45], Training Loss: 0.0029, Training F1 Score: 0.6623
Epoch [5/45], Validation Loss: 0.0027, Validation F1 Score: 0.7115
Epoch [6/45], Training Loss: 0.0026, Training F1 Score: 0.7347
Epoch [6/45], Validation Loss: 0.0024, Validation F1 Score: 0.7676
Epoch [7/45], Training Loss: 0.0024, Training F1 Score: 0.7811
Epoch [7/45], Validation Loss: 0.0022, Validation F1 Score: 0.8089
Epoch [8/45], Training Loss: 0.0022, Training F1 Score: 0.8154
Epoch [8/45], Validation Lo

In [10]:
with open('ATE_test.json', 'r') as file:
    valset = json.load(file)

print(len(valset))
max_seq_len = 85

word_embeddings = []
labels = []

for key, value in valset.items():
    text = value['text']
    label_seq = value['labels']

    label_list = list(label_map)  # Store index of each 27 classes created
    label_indices = [label_list.index(label) for label in label_seq]
    label_indices_padded = label_indices[:max_seq_len] + [0] * (max_seq_len - len(label_indices))
    labels.append(label_indices_padded)

    vectors = [word2vec_model[word] if word in word2vec_model else np.zeros(300) for word in text]
    vectors_padded = [np.pad(vec, (0, 300), constant_values=0)[:300] for vec in vectors]
    vectors_padded += [np.zeros(300)] * (max_seq_len - len(vectors_padded))
    word_embeddings.append(vectors_padded)

word_embeddings_array = np.array(word_embeddings)

lol_texts_tensor = torch.tensor(word_embeddings_array, dtype=torch.float32)
lol_labels_tensor = torch.tensor(labels)

with torch.no_grad():
    model = LSTMTagger(input_size, hidden_size, output_size)
    model = model.to(device)
    model.load_state_dict(torch.load('t2_lstm_word2vec.pt'))
    lol_outputs = model(lol_texts_tensor.to(device)) 
    argmax_indices = torch.argmax(lol_outputs, dim=2)
    reshaped_tensor = argmax_indices.view(328, 85)
    fl_out = reshaped_tensor.flatten()
    fl_label = lol_labels_tensor.flatten().to(device) 
    f1 = f1_score(fl_out.cpu(), fl_label.cpu(), average='macro') 
    print("F1 Score on Test Set:", f1)
    accuracy = accuracy_score(fl_out.cpu(), fl_label.cpu())
    print("Accuracy on Test Set:", accuracy)

328
F1 Score on Test Set: 0.892654462700401
Accuracy on Test Set: 0.9826040172166427


In [ ]:
# wandb.init(
#     project="nlp_ass2B", 
#     name=f"GRU_Word2Vec"
# )

In [12]:
class GRUTagger(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUTagger, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc1(out)
        return out
    
input_size = 300
hidden_size = 256
output_size = 3

model = GRUTagger(input_size, hidden_size, output_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 45
batch_size = 32 

texts_tensor = texts_tensor.to(device)
labels_tensor = labels_tensor.to(device)
val_texts_tensor = test_texts_tensor
val_labels_tensor =  test_labels_tensor
val_texts_tensor = val_texts_tensor.to(device)
val_labels_tensor = val_labels_tensor.to(device)

save_loss = 0

for epoch in range(num_epochs):
    
    model.train()  
    total_loss=0
    train_predictions = []
    train_labels = []
    for i in range(0, len(texts_tensor), batch_size):
        optimizer.zero_grad()
        batch_texts = texts_tensor[i:i+batch_size]
        batch_labels = labels_tensor[i:i+batch_size].view(-1)
        outputs = model(batch_texts)
        loss = criterion(outputs.view(-1, output_size), batch_labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        train_predictions.extend(torch.argmax(outputs, dim=2).flatten().cpu().tolist())
        train_labels.extend(batch_labels.cpu().tolist())
        
    train_f1 = f1_score(train_labels, train_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_loss/len(texts_tensor):.4f}, Training F1 Score: {train_f1:.4f}')
    
    # Validation
    model.eval()  
    total_val_loss = 0
    val_predictions = []
    val_labels = []
    
    with torch.no_grad():
        for i in range(0, len(val_texts_tensor), batch_size):
            batch_texts_val = val_texts_tensor[i:i+batch_size]
            batch_labels_val = val_labels_tensor[i:i+batch_size].view(-1)
            outputs_val = model(batch_texts_val)
            val_loss = criterion(outputs_val.view(-1, output_size), batch_labels_val)
            total_val_loss += val_loss.item()
            val_predictions.extend(torch.argmax(outputs_val, dim=2).flatten().cpu().tolist())
            val_labels.extend(batch_labels_val.cpu().tolist())
    
    val_f1 = f1_score(val_labels, val_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {total_val_loss/len(val_texts_tensor):.4f}, Validation F1 Score: {val_f1:.4f}')
    if(val_f1>save_loss):
        save_loss = val_f1
        torch.save(model.state_dict(),'t2_gru_word2vec.pt')
    log_metric = {"Epoch": epoch+1, "Training Loss": total_loss/len(texts_tensor), "Training F1 Score": train_f1, "Validation Loss": total_val_loss/len(val_texts_tensor), "Validation F1 Score": val_f1}
    # wandb.log(log_metric)
print("Finished Training")
    

Epoch [1/45], Training Loss: 0.0134, Training F1 Score: 0.5973
Epoch [1/45], Validation Loss: 0.0048, Validation F1 Score: 0.6267
Epoch [2/45], Training Loss: 0.0040, Training F1 Score: 0.6359
Epoch [2/45], Validation Loss: 0.0036, Validation F1 Score: 0.6393
Epoch [3/45], Training Loss: 0.0034, Training F1 Score: 0.6471
Epoch [3/45], Validation Loss: 0.0031, Validation F1 Score: 0.6646
Epoch [4/45], Training Loss: 0.0030, Training F1 Score: 0.6998
Epoch [4/45], Validation Loss: 0.0028, Validation F1 Score: 0.7509
Epoch [5/45], Training Loss: 0.0027, Training F1 Score: 0.7644
Epoch [5/45], Validation Loss: 0.0025, Validation F1 Score: 0.7928
Epoch [6/45], Training Loss: 0.0024, Training F1 Score: 0.7965
Epoch [6/45], Validation Loss: 0.0023, Validation F1 Score: 0.8237
Epoch [7/45], Training Loss: 0.0022, Training F1 Score: 0.8290
Epoch [7/45], Validation Loss: 0.0021, Validation F1 Score: 0.8380
Epoch [8/45], Training Loss: 0.0021, Training F1 Score: 0.8450
Epoch [8/45], Validation Lo

In [13]:
with open('ATE_test.json', 'r') as file:
    valset = json.load(file)

print(len(valset))
max_seq_len = 85

word_embeddings = []
labels = []

for key, value in valset.items():
    text = value['text']
    label_seq = value['labels']

    label_list = list(label_map)  # Store index of each 27 classes created
    label_indices = [label_list.index(label) for label in label_seq]
    label_indices_padded = label_indices[:max_seq_len] + [0] * (max_seq_len - len(label_indices))
    labels.append(label_indices_padded)

    vectors = [word2vec_model[word] if word in word2vec_model else np.zeros(300) for word in text]
    vectors_padded = [np.pad(vec, (0, 300), constant_values=0)[:300] for vec in vectors]
    vectors_padded += [np.zeros(300)] * (max_seq_len - len(vectors_padded))
    word_embeddings.append(vectors_padded)

word_embeddings_array = np.array(word_embeddings)

lol_texts_tensor = torch.tensor(word_embeddings_array, dtype=torch.float32)
lol_labels_tensor = torch.tensor(labels)

with torch.no_grad():
    model = GRUTagger(input_size, hidden_size, output_size)
    model = model.to(device)
    model.load_state_dict(torch.load('t2_gru_word2vec.pt'))
    lol_outputs = model(lol_texts_tensor.to(device)) 
    argmax_indices = torch.argmax(lol_outputs, dim=2)
    reshaped_tensor = argmax_indices.view(328, 85)
    fl_out = reshaped_tensor.flatten()
    fl_label = lol_labels_tensor.flatten().to(device) 
    f1 = f1_score(fl_out.cpu(), fl_label.cpu(), average='macro') 
    print("F1 Score on Test Set:", f1)
    accuracy = accuracy_score(fl_out.cpu(), fl_label.cpu())
    print("Accuracy on Test Set:", accuracy)

328
F1 Score on Test Set: 0.8941870888500812
Accuracy on Test Set: 0.9818507890961262
